In [1]:
import pickle

file = open('/home/tom/work/rltensor/rltensor/app/ticker1.pkl', 'rb')
tickers = pickle.loads(file.read())
print(tickers)

['USDT_BCH', 'USDT_BTC', 'USDT_DASH', 'USDT_ETC', 'USDT_ETH', 'USDT_LTC', 'USDT_XMR', 'USDT_ZEC']


from pytrade_env.database.utils import get_data
import pandas as pd


start = '2018-01-01 00:00:00'
end = '2018-09-01 00:00:00'
dfs = dict()
index = None
tickers = tickers
for ticker in tickers:
    df = get_data(ticker, start, end, period=1800)
    df = df.dropna()
    df_ = df[['high', 'low', 'close', 'open', 'volume']]
    time_idx = pd.DatetimeIndex(df['date'])
    df_.index = time_idx
    dfs[ticker] = df_
    if index is None:
        index = df_.index
    else:
        index = index & df_.index
        
for ticker in tickers:
    dfs[ticker] = dfs[ticker].loc[index]
    
for ticker in tickers:
    dfs[ticker].to_csv(f'../datasets/crypto/{ticker}.csv')

In [2]:
import pandas as pd
dfs = dict()
for ticker in tickers:
    df = pd.read_csv(f'../datasets/crypto/{ticker}.csv')
    df.index = pd.DatetimeIndex(df['date'])
    del df['date']
    dfs[ticker] = df

In [3]:
%%time
import backtrader as bt

from btbot import get_cerebro
from btbot.strategies import RLStrategy
from rl_traders.agents import EIIEAgent

cerebro = get_cerebro(startcash=10000)

for ticker in tickers:
    df = dfs[ticker]
    cols = list(df.columns)
    close_idx = cols.index('close')
    high_idx = cols.index('high')
    low_idx = cols.index('low')
    open_idx = cols.index('open')
    volume_idx =cols.index('volume')
    stock = bt.feeds.PandasData(dataname=df, close=close_idx, high=high_idx,
                                low=low_idx, open=open_idx, volume=volume_idx)
    cerebro.adddata(stock, name=ticker)
    
action_spec = {'shape': len(tickers) + 1, 'type': 'continuous'}

LR_SPEC = {"lr": 3.0e-3, 'name': 'adam'}
SCHEDULER_SPEC = {'name': 'plateau', 'patience': 1000, 'factor': 0.96, 'min_lr': 3.0e-5}
# SCHEDULER_SPEC = {'name': 'step', 'patience': 100, 'factor': 0.5, 'min_lr': 3.0e-5}
agent = EIIEAgent(action_spec,
                 is_debug=False, memory_limit=100000, window_length=50,
                 batch_size=50, beta=5.0e-5, lr_spec=LR_SPEC, scheduler_spec=SCHEDULER_SPEC, log_hist_freq=None)

cerebro.addstrategy(RLStrategy, train_freq=1, debug=False,
                    warmup=10000, agent=agent, train_n_epochs=1000000, test_n_epochs=30,
                    tickers=tickers, keys=['close', 'high', 'low', 'volume'])
strategy = cerebro.run()[0]

Delete old tensorboard log


KeyboardInterrupt: 